In [3]:
import os
import sys
import gzip
import math
import copy
import numpy
import pandas
import matplotlib

import numpy as np

import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from scipy.ndimage import gaussian_filter



samples = ["HG00621", "HG00741", "HG01952", "HG01978", "HG03516"]
replicates = [1, 2]
phases = ["mat", "pat"]



In [ ]:
class FastaReader(object):

    def __init__(self):
        self.data = {}
        return

    def read(self, fastafile):
        chrom = ""
        seq = ""

        for l in open(fastafile):
            l = l.strip()

            if l.startswith(">"):
                if chrom != "":
                    self.data[chrom] = seq

                chrom = l[1:]
                seq = ""

            else:
                seq += l.lower()

        self.data[chrom] = seq

        return

    def chromosomes(self):
        return set(self.data.keys())

    def get(self, chrom, start, end, zero_based=False, include_end=True):
        if not zero_based:
            start -= 1
            end -= 1
        if include_end:
            end += 1

        return self.data[chrom][start:end]


hg38_fasta_fp = f"/scratch/wzhang/ref/linear/hg38/bismark/hg38.fasta"
hg38_fasta = FastaReader()
hg38_fasta.read(hg38_fasta_fp)










In [13]:
chroms = hg38_fasta.chromosomes()
cpg_sites = {}

for chrom in chroms:
    chrom_short = chrom.split()[0]
    seq = hg38_fasta.data[chrom].upper()
    
    cpg_sites_this_chrom = set()
    for i in range(len(seq)-1):

        if seq[i: i+2] in ["CG"]:
            cpg_sites_this_chrom.add(i+1)
    
    cpg_sites[chrom_short] = cpg_sites_this_chrom
    
    
    # print(chrom_short, len(seq), len(cpg_sites_this_chrom), len(cpg_sites_this_chrom)/len(seq))
    
    



KeyboardInterrupt



In [12]:
cpg_ref_fp = "/scratch/wzhang/ref/linear/hg38/bismark/cpg_sites.bed"
cpg_ref_fh = open(cpg_ref_fp, "w")


for chrom in sorted(cpg_sites.keys()):
    for pos in sorted(cpg_sites[chrom]):
        line = f"{chrom}\t{pos}\t{pos+2}\n"
        cpg_ref_fh.write(line)

cpg_ref_fh.close()
    

In [4]:
# change name


# Generate PerCpG methylation
# prepare methylC for both types
Ω

# bismark_hg38_fu 
# bismark_hg38_default
run_para_folder = "bismark_hg38_default"

for s in samples:
    for r in replicates:
        
        print(s, r)
        wd = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/"

        f1i = f"{wd}/bismark.cov.gz"
        f1o = f"{wd}/bismark.bg"
        
        f2i = f"{wd}/bismark.cov.gz.bismark.cov.gz"
        f2o1 = f"{wd}/track.cytosine.methylc"
        f2o2 = f"{wd}/track.cpg.methylc"
        
        f2o1_fh = open(f2o1, "w")
        f2o2_fh = open(f2o2, "w")
        
        
        
        data = {}
        i = 1
        for l in gzip.open(f2i, "rt"):
            l = l.strip().split()
            chrom, start, end, ml, met, unmet = l
            
            start = int(start)
            met = int(met)
            unmet = int(unmet)
            ml = float(ml) /100
            cov = met + unmet
            
            if chrom not in data:
                data[chrom] = {}
            
            strand = "+"
            if start not in cpg_sites[chrom]:
                strand = "-"
            
            data[chrom][start] = (met, cov)

            
            f2o1_fh.write(f"{chrom}\t{start-1}\t{start}\tCG\t{ml}\t{strand}\t{cov}\n")
        
        f2o1_fh.close()
        
        
        
        cpg_sites[chrom_short] = cpg_sites_this_chrom
        
        for chrom in cpg_sites:
            for start in sorted(cpg_sites[chrom]):
                if chrom not in data:
                    continue
                
                c1 = (0, 0)
                c2 = (0, 0)
                
                if start in data[chrom]:
                    c1 = data[chrom][start]
                    del data[chrom][start]
                
                if start+1 in data[chrom]:
                    c2 = data[chrom][start+1]
                    del data[chrom][start+1]
                
                met = c1[0] + c2[0]
                cov = c1[1] + c2[1]
                
                if cov == 0:
                    continue
                
                ml = met / cov
                
                f2o2_fh.write(f"{chrom}\t{start-1}\t{start+1}\tCG\t{ml}\t+\t{cov}\n")
        
        

        f2o2_fh.close()
        

            
            
            




HG00621 1
HG00621 2
HG00741 1
HG00741 2
HG01952 1
HG01952 2
HG01978 1
HG01978 2
HG03516 1
HG03516 2


In [14]:
# clean methylGrapher lifted result, and merge replicate




for s in samples:
    print(s)
    
    
    
    for r in replicates:
        print(r)

        fin_mg2l = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan_lifted/{s}_{r}.methylc.gz"
        
        data_mg2l = {}
        for l in gzip.open(fin_mg2l, "rt"):
            l = l.strip().split()
            # print(l)
            chrom, start, end, context, ml, strand, cov, xxx = l
            
            
            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)
            
            met = int(round(ml * cov))
            
            if chrom not in data_mg2l:
                data_mg2l[chrom] = {}
            
            if start not in data_mg2l[chrom]:
                data_mg2l[chrom][start] = [0, 0]
            
            data_mg2l[chrom][start][0] += met
            data_mg2l[chrom][start][1] += cov
        
        
    
        fout_mg2l = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan_lifted/{s}_{r}.cytosine.methylc"
        fhout_mg2l = open(fout_mg2l, "w")
        for chrom in sorted(data_mg2l.keys()):
            for start in sorted(data_mg2l[chrom]):
                met, cov = data_mg2l[chrom][start]
                ml = met / cov
                fhout_mg2l.write(f"{chrom}\t{start}\t{start+1}\tCG\t{ml}\t+\t{cov}\n")
        fhout_mg2l.close()
    
    


for s in samples:
    print(s)
    
    data_mg2l = {}
    for r in replicates:
        print(r)

        fin_mg2l = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan_lifted/{s}_{r}.methylc.gz"
        for l in gzip.open(fin_mg2l, "rt"):
            l = l.strip().split()
            # print(l)
            chrom, start, end, context, ml, strand, cov, xxx = l
            
            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)
            
            met = int(round(ml * cov))
            
            if chrom not in data_mg2l:
                data_mg2l[chrom] = {}
            
            if start not in data_mg2l[chrom]:
                data_mg2l[chrom][start] = [0, 0]
            
            data_mg2l[chrom][start][0] += met
            data_mg2l[chrom][start][1] += cov
        
        
    
    fout_mg2l = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan_lifted_merge_replicate/{s}.cytosine.methylc"
    fhout_mg2l = open(fout_mg2l, "w")
    for chrom in sorted(data_mg2l.keys()):
        for start in sorted(data_mg2l[chrom]):
            met, cov = data_mg2l[chrom][start]
            ml = met / cov
            fhout_mg2l.write(f"{chrom}\t{start}\t{start+1}\tCG\t{ml}\t+\t{cov}\n")
    fhout_mg2l.close()
    

        
        
        

HG00621
1
2
HG00741
1
2
HG01952
1
2
HG01978
1
2
HG03516
1
2
HG00621
1
2
HG00741
1
2
HG01952
1
2
HG01978
1
2
HG03516
1
2


In [5]:
# Merge replicate for bismark (cytosine and cpg)

for s in samples:
    print(s)
    
    data_bs_cytosine = {}
    data_bs_cpg = {}
    
    for r in replicates:
        print(r)
        
        
        fin_bs = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/track.cytosine.methylc"
        for l in open(fin_bs):
            l = l.strip().split()
            # print(l)
            chrom, start, end, context, ml, strand, cov = l
            
            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)
            
            met = int(round(ml * cov))
            
            if chrom not in data_bs_cytosine:
                data_bs_cytosine[chrom] = {}
            
            if start not in data_bs_cytosine[chrom]:
                data_bs_cytosine[chrom][start] = [0, 0]
            
            data_bs_cytosine[chrom][start][0] += met
            data_bs_cytosine[chrom][start][1] += cov
            
        
        fin_bs = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/track.cpg.methylc"
        for l in open(fin_bs):
            l = l.strip().split()
            # print(l)
            chrom, start, end, context, ml, strand, cov = l
            
            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)
            
            met = int(round(ml * cov))
            
            if chrom not in data_bs_cpg:
                data_bs_cpg[chrom] = {}
            
            if start not in data_bs_cpg[chrom]:
                data_bs_cpg[chrom][start] = [0, 0]
            
            data_bs_cpg[chrom][start][0] += met
            data_bs_cpg[chrom][start][1] += cov
    
    fout_bs = f"/scratch/wzhang/projects/ggWGBS/data/bismark_hg38_merge_replicate/{s}.cytosine.methylc"
    fhout_bs = open(fout_bs, "w")
    for chrom in sorted(data_bs_cytosine.keys()):
        for start in sorted(data_bs_cytosine[chrom]):
            met, cov = data_bs_cytosine[chrom][start]
            ml = met / cov
            fhout_bs.write(f"{chrom}\t{start}\t{start+1}\tCG\t{ml}\t+\t{cov}\n")
    fhout_bs.close()
    
    
    fout_bs = f"/scratch/wzhang/projects/ggWGBS/data/bismark_hg38_merge_replicate/{s}.cpg.methylc"
    fhout_bs = open(fout_bs, "w")
    for chrom in sorted(data_bs_cpg.keys()):
        for start in sorted(data_bs_cpg[chrom]):
            met, cov = data_bs_cpg[chrom][start]
            ml = met / cov
            fhout_bs.write(f"{chrom}\t{start}\t{start+2}\tCG\t{ml}\t+\t{cov}\n")
    fhout_bs.close()


HG00621
1
2
HG00741
1
2
HG01952
1
2
HG01978
1
2
HG03516
1
2


In [5]:
# Merge replicate for methylGrapher



for s in samples:
    print(s)
    data_mg = {}
    
    for r in replicates:
        print(r)
        
        fin_mg = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan/{s}_{r}.cytosine.tsv"
        for l in open(fin_mg):
            l = l.strip().split()
            segID, pos, strand, context, unmet, met, cov, ml = l
            
            if segID not in data_mg:
                data_mg[segID] = {}
            
            if pos not in data_mg[segID]:
                data_mg[segID][pos] = [0, 0]
            
            data_mg[segID][pos][0] += int(met)
            data_mg[segID][pos][1] += int(cov)

    
    fout_mg = f"/scratch/wzhang/projects/ggWGBS/data/methylgrapher_pan_merge_replicate/{s}.cytosine.methyl"
    fhout_mg = open(fout_mg, "w")
    for segID in data_mg.keys():
        for pos in sorted(data_mg[segID]):
            met, cov = data_mg[segID][pos]
            ml = met / cov
            fhout_mg.write(f"{segID}\t{pos}\t+\tCG\t{cov - met}\t{met}\t{cov}\t{ml}\n")
    fhout_mg.close()
        
        

HG00621
1
2
HG00741
1
2
HG01952
1
2
HG01978
1
2
HG03516
1
2


In [ ]:
# ----------- Divider ------------

# Prepare tracks



In [6]:
# Prepare tracks for hg38

for s in samples:
    for r in replicates:
        
        wd = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/"
        
        f2o1 = f"{wd}/track.cytosine.methylc"
        f2o2 = f"{wd}/track.cpg.methylc"
        
        for fo in [f2o1, f2o2]:
            
            cmd1 = f"sort -k1,1 -k2,2n -o {fo} {fo}"
            cmd2 = f"bgzip {fo}"
            cmd3 = f"tabix -p bed {fo}.gz"
            cmd4 = f"cat {fo}.gz | gunzip > {fo}"
            
            for cmd in [cmd1, cmd2, cmd3, cmd4]:
                print(cmd)
                
        print()
    print()
        
        

sort -k1,1 -k2,2n -o /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc
bgzip /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc
tabix -p bed /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc.gz
cat /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc.gz | gunzip > /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cytosine.methylc
sort -k1,1 -k2,2n -o /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cpg.methylc /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cpg.methylc
bgzip /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cpg.methylc
tabix -p bed /scratch/wzhang/projects/ggWGBS/bismark_hg38_default/HG00621/1//track.cpg.methylc.gz
cat /scratch/wzhang/projects/ggWGBS/bismark_hg

In [ ]:
# Make symbolic links to the data folder




for s in samples:
    for r in replicates:
        
        wd = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/"
        
        f2o1 = f"cytosine.methylc"
        f2o2 = f"cpg.methylc"
        
        for fo in [f2o1, f2o2]:
            cmd = f"ln -s {wd}/track.{fo} /scratch/wzhang/projects/ggWGBS/data/bismark_hg38/{s}_{r}.{fo}"
            
            print(cmd)






In [11]:
# Prepare tracks for individual samples

for run_para_folder in ["bismark_ind_default", "bismark_ind_fu"]:

    for s in samples:
        for r in replicates:
            for p in phases:
            
                wd = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/{p}/"
                # print(wd)
                
                
                f2o1 = f"{wd}/track.cytosine.methylc"
                f2o2 = f"{wd}/track.cpg.methylc"
                
                for fo in [f2o1, f2o2]:
                    
                    cmd1 = f"sort -k1,1 -k2,2n -o {fo} {fo}"
                    cmd2 = f"bgzip {fo}"
                    cmd3 = f"tabix -p bed {fo}.gz"
                    cmd4 = f"cat {fo}.gz | gunzip > {fo}"
                    for cmd in [cmd1, cmd2, cmd3, cmd4]:
                        print(cmd)
                    print()
                    
            print()
        
        break
        print()
      

sort -k1,1 -k2,2n -o /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc
bgzip /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc
tabix -p bed /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc.gz
cat /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc.gz | gunzip > /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cytosine.methylc

sort -k1,1 -k2,2n -o /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cpg.methylc /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cpg.methylc
bgzip /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cpg.methylc
tabix -p bed /scratch/wzhang/projects/ggWGBS/bismark_ind_default/HG00621/1/mat//track.cpg.methylc.gz
cat /scratch/wz

In [ ]:
# ----------- Divider ------------

# Debug code from this point

In [20]:
class FastaReader(object):

    def __init__(self):
        self.data = {}
        return

    def read(self, fastafile):
        chrom = ""
        seq = ""

        for l in open(fastafile):
            l = l.strip()

            if l.startswith(">"):
                if chrom != "":
                    self.data[chrom] = seq

                chrom = l[1:]
                seq = ""

            else:
                seq += l.lower()

        self.data[chrom] = seq

        return

    def chromosomes(self):
        return set(self.data.keys())

    def get(self, chrom, start, end, zero_based=False, include_end=True):
        if not zero_based:
            start -= 1
            end -= 1
        if include_end:
            end += 1

        return self.data[chrom][start:end]



def get_cpg_sites(fasta_fp):
    
    fasta_obj = FastaReader()
    fasta_obj.read(fasta_fp)

    chroms = fasta_obj.chromosomes()
    cpg_sites = {}
    
    for chrom in chroms:
        chrom_short = chrom.split()[0]
        seq = fasta_obj.data[chrom].upper()
        
        cpg_sites_this_chrom = set()
        for i in range(len(seq)-1):
    
            if seq[i: i+2] in ["CG"]:
                cpg_sites_this_chrom.add(i+1)
        
        cpg_sites[chrom_short] = cpg_sites_this_chrom
    return cpg_sites
 
 
 
 
 
 
for run_para_folder in ["bismark_ind_default", "bismark_ind_fu"]:

    for s in samples:
        
        mat_genome = f"/scratch/wzhang/ref/linear/pan/{s}_mat/genome.fasta"
        pat_genome = f"/scratch/wzhang/ref/linear/pan/{s}_pat/genome.fasta"
        
        mat_cpg_sites = get_cpg_sites(mat_genome)
        pat_cpg_sites = get_cpg_sites(pat_genome)
        
        
        for r in replicates:
            
            for p in phases:
            
                print(s, r, p, run_para_folder)
                
                wd = f"/scratch/wzhang/projects/ggWGBS/{run_para_folder}/{s}/{r}/{p}"
                cpg_sites = mat_cpg_sites
                if p == "pat":
                    cpg_sites = pat_cpg_sites

        
                f1i = f"{wd}/bismark.cov.gz"
                f1o = f"{wd}/bismark.bg"
                
                f2i = f"{wd}/bismark.cov.gz.bismark.cov.gz"
                f2o1 = f"{wd}/track.cytosine.methylc"
                f2o2 = f"{wd}/track.cpg.methylc"
                
                f2o1_fh = open(f2o1, "w")
                f2o2_fh = open(f2o2, "w")
                
                
                data = {}
                i = 1
                for l in gzip.open(f2i, "rt"):
                    l = l.strip().split()
                    chrom, start, end, ml, met, unmet = l
                    
                    start = int(start)
                    met = int(met)
                    unmet = int(unmet)
                    ml = float(ml) /100
                    cov = met + unmet
                    
                    if chrom not in data:
                        data[chrom] = {}
                    
                    strand = "+"
                    if start not in cpg_sites[chrom]:
                        strand = "-"
                    
                    data[chrom][start] = (met, cov)
                    
                    # TODO add this later
                    f2o1_fh.write(f"{chrom}\t{start-1}\t{start}\tCG\t{ml}\t{strand}\t{cov}\n")
                
                f2o1_fh.close()
                
                
                
                cpg_sites[chrom_short] = cpg_sites_this_chrom
                
                for chrom in cpg_sites:
                    for start in sorted(cpg_sites[chrom]):
                        if chrom not in data:
                            continue
                        
                        c1 = (0, 0)
                        c2 = (0, 0)
                        
                        if start in data[chrom]:
                            c1 = data[chrom][start]
                            del data[chrom][start]
                        
                        if start+1 in data[chrom]:
                            c2 = data[chrom][start+1]
                            del data[chrom][start+1]
                        
                        met = c1[0] + c2[0]
                        cov = c1[1] + c2[1]
                        
                        if cov == 0:
                            continue
                        
                        ml = met / cov
                        
                        f2o2_fh.write(f"{chrom}\t{start-1}\t{start+1}\tCG\t{ml}\t+\t{cov}\n")
                
        
                f2o2_fh.close()
        
        break




HG00621 1 mat bismark_ind_default
HG00621 1 pat bismark_ind_default
HG00621 2 mat bismark_ind_default
HG00621 2 pat bismark_ind_default
HG00621 1 mat bismark_ind_fu
HG00621 1 pat bismark_ind_fu
HG00621 2 mat bismark_ind_fu
HG00621 2 pat bismark_ind_fu
